In [7]:
import gradio as gr
import pennylane as qml
from pennylane import numpy as np
from sklearn import datasets
from sklearn.preprocessing import StandardScaler
from sklearn.decomposition import PCA
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score
import matplotlib.pyplot as plt

# Load iris dataset once
iris = datasets.load_iris()
X_all = iris.data
y_all = iris.target

scaler = StandardScaler()
X_all_scaled = scaler.fit_transform(X_all)

n_qubits = 4

def feature_map(x):
    for i in range(n_qubits):
        qml.RY(x[i], wires=i)

def entangling_layer(entanglement_type):
    if entanglement_type == "chain":
        for i in range(n_qubits - 1):
            qml.CNOT(wires=[i, i + 1])

def layered_circuit(x, params, depth, entanglement_type):
    for d in range(depth):
        feature_map(x)
        for i in range(n_qubits):
            qml.RY(params[d, i], wires=i)
        entangling_layer(entanglement_type)
    return [qml.expval(qml.PauliZ(i)) for i in range(n_qubits)]

def make_qnode(depth, entanglement_type):
    dev = qml.device("default.qubit", wires=n_qubits)
    
    @qml.qnode(dev)
    def circuit(x, params):
        return layered_circuit(x, params, depth, entanglement_type)
    return circuit

def make_embeddings(X, qnode, params):
    return np.array([qnode(x, params) for x in X])

def create_pca_plot(X_classical, X_quantum, y, sample_count):
    pca_classical = PCA(n_components=2)
    pca_quantum = PCA(n_components=2)
    
    X_classical_pca = pca_classical.fit_transform(X_classical)
    X_quantum_pca = pca_quantum.fit_transform(X_quantum)
    
    fig, axs = plt.subplots(1, 2, figsize=(12, 5))
    colors = ['red', 'green', 'blue']
    
    axs[0].set_title('Quantum Embeddings (PCA)')
    for i, color in enumerate(colors):
        idxs = np.where(y == i)[0][:sample_count]
        axs[0].scatter(X_quantum_pca[idxs, 0], X_quantum_pca[idxs, 1], label=iris.target_names[i], c=color, edgecolors='k', alpha=0.7)
    axs[0].set_xlabel('PC 1')
    axs[0].set_ylabel('PC 2')
    axs[0].legend()
    axs[0].grid(True, alpha=0.3)
    
    axs[1].set_title('Classical Features (PCA)')
    for i, color in enumerate(colors):
        idxs = np.where(y == i)[0][:sample_count]
        axs[1].scatter(X_classical_pca[idxs, 0], X_classical_pca[idxs, 1], label=iris.target_names[i], c=color, edgecolors='k', alpha=0.7)
    axs[1].set_xlabel('PC 1')
    axs[1].set_ylabel('PC 2')
    axs[1].legend()
    axs[1].grid(True, alpha=0.3)
    
    plt.tight_layout()
    return fig

def quantum_vs_classical_demo(sample_count, depth, entanglement_type):
    # Shuffle data to ensure multiple classes in subset
    np.random.seed(42)
    indices = np.random.permutation(len(X_all_scaled))
    X_shuffled = X_all_scaled[indices]
    y_shuffled = y_all[indices]
    
    sample_count = min(sample_count, len(X_all_scaled))
    X_sel = X_shuffled[:sample_count]
    y_sel = y_shuffled[:sample_count]
    
    # Classical logistic regression baseline
    clf_classical = LogisticRegression(max_iter=200, multi_class='auto')
    clf_classical.fit(X_sel, y_sel)
    y_pred_classical = clf_classical.predict(X_sel)
    acc_classical = accuracy_score(y_sel, y_pred_classical)
    
    # Params must reflect depth and nqbits
    np.random.seed(42)
    params = np.random.uniform(-0.5, 0.5, size=(depth, n_qubits))
    qnode = make_qnode(depth, entanglement_type)
    
    X_quantum_emb = make_embeddings(X_sel, qnode, params)
    
    clf_quantum = LogisticRegression(max_iter=200, multi_class='auto')
    clf_quantum.fit(X_quantum_emb, y_sel)
    y_pred_quantum = clf_quantum.predict(X_quantum_emb)
    acc_quantum = accuracy_score(y_sel, y_pred_quantum)
    
    fig = create_pca_plot(X_sel, X_quantum_emb, y_sel, sample_count)
    
    return fig, f"{acc_classical*100:.2f}%", f"{acc_quantum*100:.2f}%"

app = gr.Interface(
    quantum_vs_classical_demo,
    inputs=[
        gr.Slider(minimum=3, maximum=30, step=1, value=15, label="Number of Samples"),
        gr.Slider(minimum=1, maximum=3, step=1, value=2, label="Quantum Circuit Depth"),
        gr.Dropdown(choices=["none", "chain"], value="none", label="Entanglement Type"),
    ],
    outputs=[
        gr.Plot(label="PCA Comparison Plot"),
        gr.Textbox(label="Classical Logistic Regression Accuracy"),
        gr.Textbox(label="Quantum Embedding Logistic Regression Accuracy"),
    ],
    title="Quantum vs Classical Iris Embedding Visualizer",
    description=(
        "Visualize and compare PCA projections of classical Iris features and quantum embeddings "
        "from a parameterized quantum circuit on a simulator. \n"
        "Adjust circuit depth and entanglement to see how it affects clustering and classification accuracy."
    ),
    allow_flagging="never"
)

if __name__ == "__main__":
    app.launch(server_name="0.0.0.0", server_port=45903)


/usr/local/python/3.12.1/lib/python3.12/site-packages/gradio/interface.py:415: UserWarning: The `allow_flagging` parameter in `Interface` is deprecated. Use `flagging_mode` instead.
  warnings.warn(


* Running on local URL:  http://0.0.0.0:45903
* To create a public link, set `share=True` in `launch()`.


/home/codespace/.local/lib/python3.12/site-packages/sklearn/linear_model/_logistic.py:1264: FutureWarning: 'multi_class' was deprecated in version 1.5 and will be removed in 1.7. From then on, it will always use 'multinomial'. Leave it to its default value to avoid this warning.
  warnings.warn(
/home/codespace/.local/lib/python3.12/site-packages/sklearn/linear_model/_logistic.py:1264: FutureWarning: 'multi_class' was deprecated in version 1.5 and will be removed in 1.7. From then on, it will always use 'multinomial'. Leave it to its default value to avoid this warning.
  warnings.warn(
/home/codespace/.local/lib/python3.12/site-packages/sklearn/linear_model/_logistic.py:1264: FutureWarning: 'multi_class' was deprecated in version 1.5 and will be removed in 1.7. From then on, it will always use 'multinomial'. Leave it to its default value to avoid this warning.
  warnings.warn(
/home/codespace/.local/lib/python3.12/site-packages/sklearn/linear_model/_logistic.py:1264: FutureWarning: 'm